### LTV Prediction

* Imports


In [1]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
# import seaborn as sns


import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.express as px

In [2]:
from time import perf_counter 


In [3]:
import time
import concurrent.futures

In [4]:
import sys


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR


* Functions

In [6]:
def print_cols(cno, cols):
    print(f'Customer: {cno}')
    cols = ['Customer No.']+cols
    print(invoice[invoice['Customer No.']==cno][cols])

* Code

In [7]:
invoice = pd.read_csv('data/invoice_refined.csv',low_memory=False)

In [8]:
cno_freq = pd.read_csv('data/cno_freq.csv',low_memory=False)

In [9]:
invoice_smm_average = pd.read_csv('data/invoice_smm_average.csv',low_memory=False)

In [10]:
inv_kms_lifetime = pd.read_csv('data/inv_kms_lifetime.csv',low_memory=False)

In [11]:
inv_snmm_avg = pd.read_csv('data/inv_snmm_avg.csv',low_memory=False)

In [12]:
for df in [invoice,cno_freq,invoice_smm_average,inv_kms_lifetime]:   
    print(df.info())
df=None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492314 entries, 0 to 492313
Data columns (total 41 columns):
Unnamed: 0             492314 non-null int64
Area / Locality        468424 non-null object
CITY                   492313 non-null object
Cash /Cashless Type    27420 non-null object
Claim No.              4804 non-null object
Cust Type              492314 non-null object
Customer No.           492314 non-null object
District               492314 non-null object
Expiry Date            22231 non-null object
Gate Pass Date         40915 non-null object
Gate Pass Time         492314 non-null object
Invoice Date           492314 non-null object
Invoice No             492314 non-null int64
Invoice Time           492314 non-null object
Job Card No            492314 non-null int64
JobCard Date           492314 non-null object
JobCard Time           492314 non-null object
KMs Reading            492314 non-null int64
Labour Total           492314 non-null float64
Make                   

In [13]:
inv_kms_lifetime[inv_kms_lifetime['KMs_3L_cap'].between(250000,300000, inclusive=True)].shape

(3419, 7)

In [14]:
invoice['Frequency_year'] = cno_freq['Frequency_year']

In [15]:
invoice['SMM_average'] = invoice_smm_average['SMM_average']

In [16]:
invoice['kms_lifetime_3L'] = inv_kms_lifetime['kms_lifetime_3L']

In [17]:
# inv_snmm_avg
invoice['SnMM_average'] = inv_snmm_avg['SnMM_average']

In [18]:
state_map={'Maharashtra': 'West',
 'Tamil Nadu': 'South',
 'Karnataka': 'South',
 'Uttar Pradesh': 'North',
 'Gujarat': 'West',
 'Telangana': 'South',
 'Punjab': 'North',
 'Andhra Pradesh': 'South',
 'Haryana': 'North',
 'Madhya Pradesh': 'West',
 'Rajasthan': 'West',
 'Kerala': 'South',
 'Himachal Pradesh': 'North',
 'Bihar': 'East',
 'Uttarakhand': 'North',
 'Odisha': 'East',
 'Puducherry': 'South',
 'West Bengal': 'East',
 'Chhattisgarh': 'East',
 'Jharkhand': 'East',
 'Assam': 'East',
 'Chandigarh': 'North'}
invoice['state_news'] = invoice['plant_state'].map(state_map)

In [19]:
invoice['state_news'].value_counts(dropna=False)

South    214484
West     183147
North     86793
East       7890
Name: state_news, dtype: int64

In [20]:
# Data usage stats

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('invoice', 856946827),
 ('inv_snmm_avg', 166771685),
 ('invoice_smm_average', 136439254),
 ('inv_kms_lifetime', 111287066),
 ('cno_freq', 38625526),
 ('state_map', 1184),
 ('SVR', 1056),
 ('make_subplots', 136),
 ('print_cols', 136),
 ('train_test_split', 136),
 ('go', 80),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('px', 80),
 ('perf_counter', 72),
 ('df', 16)]

In [21]:
# dump df other than invoice

cno_freq = None
invoice_smm_average = None
inv_kms_lifetime = None
inv_snmm_avg = None

In [22]:
invoice[invoice.Model.isnull()].shape

(1525, 46)

In [23]:
invoice.shape

(492314, 46)

In [24]:
invoice = invoice[invoice.Model.notnull()]

In [25]:
invoice.shape[0]

490789

---

In [26]:
temp = invoice[(invoice['SMM_average']<100.0)]
print(temp.shape)
print(temp[(temp['SMM_average']==temp['SMM_average']\
                                       .min())][['Customer No.', 'SMM_average']].head())
# fig = px.histogram(x=invoice['SMM_average'], barmode='overlay',marginal='box', nbins=100)
# fig.show()

(32999, 46)
      Customer No.  SMM_average
6899         86054          0.0
21926        96897          0.0
35571        87660          0.0
41421       105176          0.0
42200       105935          0.0


In [27]:
temp = invoice[(invoice['SnMM_average']<100.0)]
print(temp.shape)
print(temp[(temp['SnMM_average']==temp['SMM_average']\
                                       .min())][['Customer No.', 'SMM_average']].head())
# fig = px.histogram(x=invoice['SnMM_average'], barmode='overlay',marginal='box', nbins=100)
# fig.show()

(6951, 46)
      Customer No.  SMM_average
45981       103511          0.0
46337       102814          0.0
66926       108830          0.0
66933       112056          0.0
66977       117682          0.0


In [28]:
print_cols('145058', ['Make', 'Model','plant_city','plant_state', 'Total Value','SMM_average'])

Customer: 145058
       Customer No.                  Make          Model plant_city  \
124031       145058  MAHINDRA &  MAHINDRA  BOLERO PIK UP      CHOMU   
310383       145058  MAHINDRA &  MAHINDRA  BOLERO PIK UP      CHOMU   
342736       145058  MAHINDRA &  MAHINDRA  BOLERO PIK UP      CHOMU   

       plant_state  Total Value  SMM_average  
124031   Rajasthan       199.14         0.61  
310383   Rajasthan         0.00         0.61  
342736   Rajasthan         0.00         0.61  


In [29]:
invoice[(invoice['SMM_average']<=1000.0)].shape

(57129, 46)

In [30]:
invoice['SnMM_average_no0'] = invoice['SnMM_average']

In [31]:
invoice.loc[invoice['SnMM_average']<100.0,'SnMM_average_no0'] = 100.0

In [32]:
invoice[invoice['SnMM_average_no0']>=100.0].shape[0]+invoice.SnMM_average_no0.isnull().sum() == invoice.shape[0]

True

In [33]:
#checks out

In [34]:
temp = invoice[(invoice['kms_lifetime_3L']<.1)]
print(temp.shape)

.1*300000 #KMs life left

(1618, 47)


30000.0

In [35]:
invoice['kms_lifetime_3L_no0'] = invoice['kms_lifetime_3L']

In [36]:
invoice.loc[invoice['kms_lifetime_3L']<0.1,'kms_lifetime_3L_no0'] = 0.1

In [37]:
invoice[invoice['kms_lifetime_3L_no0']>=0.1].shape[0] == invoice.shape[0]

True

In [38]:
temp = invoice.Model.value_counts()
# k=0
# for i in temp.index:
#     print(f'{k}-{i}:{temp[i]}')
#     k +=1
model_list160 = temp[:160].index

In [39]:
invoice['Model_other'] = invoice['Model'].map(lambda x: x if x in model_list160 else 'Other')

In [40]:
invoice['Model_other'].value_counts()

SWIFT              31899
ALTO               28569
SANTRO             24536
SCORPIO CRDE       23771
WAGONR             22864
I10                21892
INDICA             19810
SWIFT DZIRE        16004
BOLERO             15909
XYLO               13346
I20                12896
INNOVA             11470
CITY               10665
VISTA              10341
INDIGO             10337
800                10185
VERNA               8564
FIGO                7709
FIESTA              7667
LOGAN D             7353
SPARK               7270
RITZ                7075
BEAT                6881
OMNI                6415
ACCENT              5734
VERITO              5573
ZEN                 5137
LOGAN P             5063
INDIGO MANZA        4990
XUV 500             4748
                   ...  
RHINO RX              87
CRETA                 86
YETI                  82
E-CLASS               82
GRAND VITARA          81
SONATA V-6            79
RIO                   78
BOLT                  77
A4                    74


In [42]:
# snmm_columns = ['plant_state','state_news','Make','Model_other',
#                 'KMs Reading','Total Value', 
#                 'Frequency_year','SnMM_average_no0','kms_lifetime_3L_no0']
snmm_columns = ['plant_state','state_news','Make','Model_other',
                'KMs Reading','Total Value', 'invoice_year',
                'Frequency_year','SnMM_average_no0','kms_lifetime_3L_no0']

In [43]:
invoice_snmm = invoice[snmm_columns]

In [44]:
invoice_snmm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490789 entries, 0 to 492313
Data columns (total 10 columns):
plant_state            490789 non-null object
state_news             490789 non-null object
Make                   490789 non-null object
Model_other            490789 non-null object
KMs Reading            490789 non-null int64
Total Value            490789 non-null float64
invoice_year           490789 non-null int64
Frequency_year         490789 non-null float64
SnMM_average_no0       490789 non-null float64
kms_lifetime_3L_no0    490789 non-null float64
dtypes: float64(4), int64(2), object(4)
memory usage: 41.2+ MB


In [46]:
# invoice_snmm.to_csv('data/invoice_snmm_LTV.csv')

In [44]:
# invoice_snmm = invoice_snmm[invoice_snmm.Model.notnull()]

In [45]:
invoice_snmm.shape

(490789, 9)

In [46]:
invoice.Make.value_counts()

MARUTI SUZUKI           146625
MAHINDRA &  MAHINDRA     87032
HYUNDAI                  85112
TATA MOTORS              57868
GENERAL MOTORS           29004
FORD                     21546
TOYOTA                   19419
HONDA                    15416
SKODA                     6500
FIAT                      6401
VOLKSWAGEN                6132
RENAULT                   3427
NISSAN                    3087
MITSUBISHI MOTORS         1427
DAEWOO                     392
BMW                        256
MERCEDES BENZ              256
AUDI                       220
HINDUSTAN MOTORS           219
FORCE                      207
PREMIER AUTOMOBILES         93
SONALIKA                    87
LAND ROVER                  21
VOLVO                       19
SAN MOTORS                   8
JEEP                         7
PORCHE                       6
MORRIS                       2
Name: Make, dtype: int64

In [47]:
invoice.Model.nunique()

245

In [86]:
month_dict = {1: 'January',
              2: 'February',
              3: 'March',
              4: 'April',
              5: 'May',
              6: 'June',
              7: 'July',
              8: 'August',
              9: 'September',
              10: 'October',
              11: 'November',
              12: 'December'}

In [48]:
# svm, xgb, rf, gb, 

In [89]:
for news in ['North', 'South', 'East', 'West']:  
    plot = invoice[invoice['state_news']==news].groupby(['state_news', 'invoice_month', 'Order Type'])\
                ['Total Value'].sum().reset_index().sort_values(['invoice_month'])
    plot['MONTH'] = plot['invoice_month'].map(month_dict)
                # .sort_values(['Invoice Date'])
#     print(plot)
    fig = px.line(plot, x="MONTH", y="Total Value", color="Order Type",
                                   line_group="Order Type")#, hover_name = "")
    fig.update_layout(
    title='Month-wise Graph for '+news+ ' India (by Order Type)')
    fig.show()

In [88]:
# plot


In [50]:
invoice_snmm.isnull().sum()

plant_state            0
state_news             0
Make                   0
Model_other            0
KMs Reading            0
Total Value            0
Frequency_year         0
SnMM_average_no0       0
kms_lifetime_3L_no0    0
dtype: int64

In [51]:
# LTV calculation

invoice_snmm['LTV_snmm'] = invoice_snmm['Frequency_year']*invoice_snmm['SnMM_average_no0']\
    *invoice_snmm['kms_lifetime_3L_no0']*100

/Users/Aniket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [52]:
invoice_snmm['LTV_snmm'] = round(invoice_snmm['LTV_snmm'],4)

/Users/Aniket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
print(invoice_snmm.LTV_snmm.min())
print(invoice_snmm.LTV_snmm.max())
print(invoice_snmm.LTV_snmm.mean())
print(invoice_snmm.LTV_snmm.median())

500.0
266848306.808
1065499.154605453
225218.14


In [54]:
5083031-1065499.154605453

4017531.845394547

---

In [55]:
X = invoice_snmm.drop(['state_news','LTV_snmm','Frequency_year',
                       'SnMM_average_no0','kms_lifetime_3L_no0'],axis=1)
X.shape

(490789, 5)

In [56]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490789 entries, 0 to 492313
Data columns (total 5 columns):
plant_state    490789 non-null object
Make           490789 non-null object
Model_other    490789 non-null object
KMs Reading    490789 non-null int64
Total Value    490789 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 22.5+ MB


In [57]:
y = invoice_snmm['LTV_snmm']
y.shape

(490789,)

In [58]:
# y

In [59]:
# one-hot encoding
X = pd.concat([X, pd.get_dummies(X.drop(columns=['KMs Reading','Total Value'],axis=1)
                                 , columns=['plant_state','Make','Model_other'])], axis=1)


In [60]:
X.drop(columns=['plant_state','Make','Model_other'], axis=1, inplace=True)

In [61]:
X.shape

(490789, 213)

In [62]:
invoice.Model_other.nunique()+invoice.Make.nunique()+invoice.plant_state.nunique()+2

213

In [63]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

In [64]:
# y_test.shape

In [65]:
# import threading
# import time

# class myThread (threading.Thread):
#    def __init__(self, threadID, name, counter):
#       threading.Thread.__init__(self)
#       self.threadID = threadID
#       self.name = name
#       self.counter = counter
#    def run(self):
#       print("Starting " + self.name)
#       # Get lock to synchronize threads
#       threadLock.acquire()
#       print_time(self.name, self.counter, 3)
#       # Free lock to release next thread
#       threadLock.release()

# def print_time(threadName, delay, counter):
# #    while counter:
# #       time.sleep(delay)
# #       print "%s: %s" % (threadName, time.ctime(time.time()))
# #       counter -= 1
#     a=99
#     b=99
#     return a,b

# threadLock = threading.Lock()
# threads = []

# # Create new threads
# thread1 = myThread(1, "Thread-1", 1)
# thread2 = myThread(2, "Thread-2", 2)

# # Start new Threads
# thread1.start()
# thread2.start()

# # Add threads to thread list
# threads.append(thread1)
# threads.append(thread2)

# # Wait for all threads to complete
# for t in threads:
#     t.join()
# print("Exiting Main Thread")

In [66]:
def fit_model(model, X_train, X_test, y_train, y_test, model_name=''):
    print(f'Executing: {model_name} ...')
    
    model_fit = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f'Completed: {model_name} ...')
    
    return model, model_fit, y_pred

In [67]:
svr_rbf = SVR(kernel='rbf', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3)

In [68]:
# t1 = time.perf_counter()

# with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
#     future1 = executor.submit(fit_model, svr_rbf, X_train, X_test, y_train, y_test, model_name='svr_rbf')
#     future2 = executor.submit(fit_model, svr_poly, X_train, X_test, y_train, y_test, model_name='svr_poly')
    
#     m_svr_rbf, svr_rbf_fit, svr_rbf_y_pred = future1.result()
#     m_svr_poly, svr_poly_fit, svr_poly_y_pred = future2.result()

# t2 = time.perf_counter()
# t2-t1

In [69]:
# # svr_rbf = SVR(kernel='rbf', C=1e3)
# # svr_poly = SVR(kernel='poly', C=1e3)

# # svr_rbf_fit = svr_rbf.fit(X, y)
# # y_rbf = svr_rbf_fit.predict(X)

# # svr_poly_fit = svr_poly.fit(X, y)
# # y_poly = svr_poly_fit.predict(X)

# #plotting
# lw = 2
# plt.scatter(X_train, y_train, color='darkorange', label='data')
# plt.plot(X_text, svr_rbf_y_pred, color='navy', lw=lw, label='RBF model')
# # plt.plot(X, y_lin, color='c', lw=lw, label='Linear model')
# plt.plot(X_text, svr_poly_y_pred, color='cornflowerblue', lw=lw, label='Polynomial model')
# plt.xlabel('data')
# plt.ylabel('target')
# plt.title('Support Vector Regression')
# plt.legend()
# plt.show()

In [70]:
stop

NameError: name 'stop' is not defined

In [ ]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror',
                        learning_rate = 0.1,
                        max_depth = 3)

In [ ]:
xg_reg.fit(X_train,y_train)

In [ ]:
ypred_xgb = xg_reg.predict(X_test)

In [ ]:
print('XGBoost:')
score = xg_reg.score(X_test, y_test)
rmse = np.sqrt(mean_squared_error(y_test, ypred_xgb))
print(f'R2:{score}\nRMSE:{rmse}')

In [91]:
print(f'Range: {y.max()}-{y.min()}')

Range: 266848306.808-500.0


In [ ]:
invoice['SMM_average_no0'].isnull().sum()

In [ ]:
invoice['SMM_average_no0'] = invoice['SMM_average']
invoice.loc[invoice['SMM_average']<100.0,'SMM_average_no0'] = 100.0

In [ ]:
y.max()/5060788.5027940655

In [ ]:
pd.qcut(y,10,range(1,11)).unique()

In [1]:
!nvidia-smi

/bin/sh: nvidia-smi: command not found
